![IES21](img/logo_ies.png)

# Situación Profesional 4: 
# Codificación de Variables Categóricas: OrdinalEncoder con Scikit-Learn
# Con Ejercicios Resueltos
## Fundamentos

In [2]:
import numpy as np
import pandas as pd

### OrdinalEncoder -  Variable categórica ordinal:  

Una variable es ordinal cuando es posible decir que uno de los valores es mayor o menor que otro, no significa que uno sea el doble o el triple del otro, pero sí que hay cierto orden entre sus valores.  Tampoco pueden sumarse o restarse.

Vamos a investigar cómo funciona OrdinalEncoder() de Scikit-Lern para esta tarea:

Docmuentación oficial: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html

~~~
sklearn.preprocessing.OrdinalEncoder(*, categories='auto', dtype=<class 'numpy.float64'>)[source]

~~~

- De la misma manera que las variables nominales, las ordinales, generalmente se presentan también con una cantidad finita de textos (por ejemplo: "Alto", "Medio", "Bajo"; o, "Primero", "Segundo", "Tercero", etc)
- Al aplicar OrdinalEncoder() con la opción de categories = 'auto', asignará un valor numérico entero comenzando en 0 hasta completar la cantidad de valores que toma la variable o columna a codificar **_ordenados alfabéticamente_**.   

Veamos el siguiente ejemplo: 

In [3]:
df1=pd.DataFrame([['m',30,'alto'],['f',25,'medio'],['f',48,'bajo'],['m',38,'alto'],
                 ['m',25,'medio'],['m',58,'medio'],['m',23,'bajo']], columns=[ 'genero', 'edad','altura'])
df1

,genero,edad,altura
0,m,30,alto
1,f,25,medio
2,f,48,bajo
3,m,38,alto
4,m,25,medio
5,m,58,medio
6,m,23,bajo


- La variable **genero** es claramente nominal, la codificaríamos con OneHotEncoder, como vimos anteriormente.   
- La variable **edad**, es claramente numérica y no es necesario codificarla de esta manera.  
- La variable **altura**, se podría tratar como nominal, **pero hay una relación de orden en sus valores (bajo < medio < alto)  y sería una pena perder esa información si la tratamos como nominal, mejor vamos a tomarla como ordinal.**  

Veamos cómo funciona con las opciones por defecto:

In [1]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer 

In [5]:
# Escriba aquí su código

tordinal1=("ordinal_enc",OrdinalEncoder(), ['altura'])

# Creamos el transformador de columnas
transformador_columnas_ordinal = ColumnTransformer(transformers=[tordinal1],remainder='passthrough')
# Lo entrenamos con fit en df1
transformador_columnas_ordinal.fit(df1)
# Lo aplicamos al df1 
ordinal1=transformador_columnas_ordinal.transform(df1)
ordinal1


array([[0.0, 'm', 30],
       [2.0, 'f', 25],
       [1.0, 'f', 48],
       [0.0, 'm', 38],
       [2.0, 'm', 25],
       [2.0, 'm', 58],
       [1.0, 'm', 23]], dtype=object)

Vemos que ha asignado valores numéricos, entre 0 y 2 ya que los valores posibles eran 3, y los asignó a los valores Alto, Medio y Bajo *ordenados alfabéticamente*.  De hecho hizo la siguiente asignación:  

alto  -> 0  
bajo  -> 1  
medio -> 2  


Como vemos, primero ordenó *alfabéticamente* los valores de altura y luego asignó los valores 0, 1 y 2 respectivamente.  



#### Ordenando los valores asignados por OrdinalEncoder()

A veces puede ser que el comportamiento anterior no sea lo más conveniente; nosotros podríamos pensar que sería mejor que los valores se asignaran **siguiendo el orden** que hemos detectado previamente:  

bajo < medio < alto ,   

de tal forma que la asignación sea:  

 
bajo   -> 0  
medio  -> 1  
alto   -> 2  


para éso nos servirá la opción **_categories_** que mencionamos anteriormente.   

Nosotros sabemos que internamente OrdinalEncoder() asignará los valores 0, 1 y 2 en ese orden, entonces le vamos a indicar que tome los valores de altura ya no en el orden alfabético, sino en el que nos conviene a nosotros, cuando definamos el OrdinalEncoder: 

In [6]:
tordinal1=("ordinal_enc",OrdinalEncoder(categories=[['bajo','medio','alto']]), ['altura'])

Le hemos indicado que queremos que el orden de asignación sea del [['bajo','medio','alto'], entonces, en ese orden asignará los valores 0,1 y 2

En cuanto al código, todo lo demás seguirá exactamente igual. 

In [7]:
# Creamos el transformador de columnas
transformador_columnas_ordinal = ColumnTransformer(transformers=[tordinal1],remainder='passthrough')
# Lo entrenamos con fit en df1
transformador_columnas_ordinal.fit(df1)
# Lo aplicamos al df1 
ordinal1=transformador_columnas_ordinal.transform(df1)
display(ordinal1)
display(df1)


array([[2.0, 'm', 30],
       [1.0, 'f', 25],
       [0.0, 'f', 48],
       [2.0, 'm', 38],
       [1.0, 'm', 25],
       [1.0, 'm', 58],
       [0.0, 'm', 23]], dtype=object)

,genero,edad,altura
0,m,30,alto
1,f,25,medio
2,f,48,bajo
3,m,38,alto
4,m,25,medio
5,m,58,medio
6,m,23,bajo


Vemos que ahora, efectivamente asignó el orden que preferíamos. 

### Ejemplo 1:

Veamos cómo proceder cuando tenemos más de una variable ordinal y en cada una de ellas deseamos imponer un orden específico, pero también tenemos otras variables ordinales a las que queremos dejar que las codifique en  el orden automático.

In [8]:
df2=pd.DataFrame([['pequeño','bajo','alto'],['grande','alto','medio'],['medio','normal','bajo'],['pequeño','alto','alto'],
                 ['medio','bajo','medio'],['grande','normal','medio'],['pequeño','alto','bajo']], columns=[ 'envergadura', 'globulos_blancos','altura'])
df2

,envergadura,globulos_blancos,altura
0,pequeño,bajo,alto
1,grande,alto,medio
2,medio,normal,bajo
3,pequeño,alto,alto
4,medio,bajo,medio
5,grande,normal,medio
6,pequeño,alto,bajo


Supongamos que tenemos dos columnas a las que queremos establecer el orden en que se asignen los valores 0, 1, 2, ..: altura (igual que antes) y globulos_blancos a la cual queremos codificar como:


bajo   -> 0  
normal -> 1  
alto   -> 2  



Podemos crear un único transformador para ellas, y le indicamos el orden de las categorías para las dos columnas de la siguiente manera:

In [9]:
tordinal2=("ordinal_enc",OrdinalEncoder(categories=[['bajo','medio','alto'],['bajo','normal','alto']]), ['altura','globulos_blancos'])


De esta manera le indicamos que:  

- para la variable altura, el orden deberá ser ['bajo','medio','alto']
- para la variable globulos_blancos el orden deberá ser: ['bajo','normal','alto']

Creamos **otro** transformador para las que queremos que se asignen **automáticamente**, en este caso la variable envergadura, el cual debería codificar de la siguiente manera: 

grande  -> 0
medio   -> 1
pequeño -> 2


In [10]:
tordinal2_auto=("ordinal_auto",OrdinalEncoder(),['envergadura'])

Ahora cuando creemos el **Transformador de Columnas** le pasaremos **los 2 transformadores**: 

In [11]:
# Creamos el transformador de columnas con los 2 transformadores: 
transformador_columnas_ordinal = ColumnTransformer(transformers=[tordinal2, tordinal2_auto],remainder='passthrough')
# Lo entrenamos con fit en df2
transformador_columnas_ordinal.fit(df2)
# Lo aplicamos al df2 
ordinal2=transformador_columnas_ordinal.transform(df2)
display(ordinal2)
display(df2)


array([[2., 0., 2.],
       [1., 2., 0.],
       [0., 1., 1.],
       [2., 2., 2.],
       [1., 0., 1.],
       [1., 1., 0.],
       [0., 2., 2.]])

,envergadura,globulos_blancos,altura
0,pequeño,bajo,alto
1,grande,alto,medio
2,medio,normal,bajo
3,pequeño,alto,alto
4,medio,bajo,medio
5,grande,normal,medio
6,pequeño,alto,bajo


Revisemos qué hizo, si no nos perdimos ... las columnas que nos brinda en el array del resultado deberían estar ordenadas de la siguiente manera:  

- altura: porque cargamos primero tordinal2 y la primer columna pasda en él es altura)  
- globulos_blancos: porque cargamos primero tordinal2 y la segunda columna pasada es globulos_blancos  
- envergadura: porque cargamos tordinal2_auto en segundo lugar  

(Francamente prefiero confiar en que lo hizo bien!)


## Inclusión de OrdinalEncoder en el Flujo de Machine Learning

Como efectuaremos estas codificaciones **después** del proceso de limpieza de missing values, nos llegará un **Train y un Test set** por separado, generados por azar, esto puede traernos problemas con los valores que toman las variables ordinales de la misma forma que nos pasó con las nominales anteriormente:

Dado que las variables ordinales toman una cantidad discreta de valores si:

- lo dejamos en categories='auto'   

correremos el riesgo que en el Train falte alguno de los valores y que por lo tanto al entrenar al transformador en el Train luego nos de un error al aplicarlo al Test set.  En ese caso deberíamos entrenarlo en todo el dataset para que reconozca todos los valores de las etiquetas como hicimos con OneHotEncoder. 

En cambio si:  

- definimos uno por uno el orden de las categorías 

no habrá error, porque habremos cargado manulmente todos los valores posibles de las categorías y sería posible indicar que entrene sólo en el Train. 

Como en un problema real podemos tener ambos tipos de situaciones, preferimos proceder de la misma manera en ambos casos tanto cuando establecemos a OrdinalEncoder en auto como cuando le pasamos el orden de cada  una de las variables:
 
> Para prevenir errores los entrenaremos en todo el dataset (igual que al OneHotEncoder).

No se preocupe que **no** hemos aprendido del Test Set, ya que de antemano conocemos las categorías que puede tener cada variable ordinal por la investigación previa que hemos hecho sobre el tema. 



### Ejemplo 2: 

Después de efectuar la limpieza de missing values, nos llega el X_train y el X_test para que codifiquemos sus variables categóricas. Dejemos todo listo para luego pasar nuestros datos a un algoritmo de Machine Learning.

In [12]:
X_train=pd.DataFrame([[50,'pequeño','normal','alto','Si'],[73,'medio','alto','medio','No'],[68,'medio','normal','bajo','No'],
                 [65,'pequeño','alto','alto', 'Si'],[76,'medio','normal','medio', 'Si'],[77,'medio','normal','medio', 'No'],
                 [52, 'pequeño','alto','bajo','Si']], 
                columns=[ 'peso','envergadura', 'globulos_blancos','altura', 'otras_enfermedades'])
X_train

,peso,envergadura,globulos_blancos,altura,otras_enfermedades
0,50,pequeño,normal,alto,Si
1,73,medio,alto,medio,No
2,68,medio,normal,bajo,No
3,65,pequeño,alto,alto,Si
4,76,medio,normal,medio,Si
5,77,medio,normal,medio,No
6,52,pequeño,alto,bajo,Si


In [13]:
X_test=pd.DataFrame([[93,'grande','alto','medio','No'],[65,'medio','normal','bajo','No'],
                 [43,'pequeño','alto','alto', 'Si'],[86,'grande','bajo','medio', 'Si']], 
                columns=[ 'peso','envergadura', 'globulos_blancos','altura', 'otras_enfermedades'])
X_test

,peso,envergadura,globulos_blancos,altura,otras_enfermedades
0,93,grande,alto,medio,No
1,65,medio,normal,bajo,No
2,43,pequeño,alto,alto,Si
3,86,grande,bajo,medio,Si


**Nota:** Adrede hemos quitado algunos valores posibles en el X_train que incluimos en el X_test, por ejemplo para  globulos_blancos el valor "normal" no figura en el X_train pero sí en el X_Test; y para envergadura no figura el valor "grande".

#### Análisis del tipo  de  Variables:
**peso**: 

- Tipo: numérica 
- Transformación: ninguna (por ahora! ...)  

**envergadura**:    

- Tipo: categórica ordinal,  
- Valores posibles: pequeño, medio, grande  
- Transformación: codificación ordinal   

**globulos_blancos**  

- Tipo: categórica ordinal,  
- Valores posibles: bajo, normal, alto  
- Transformación: codificación ordinal con asignación: bajo -> 0, normal -> 1, alto -> 2   
 

**altura**:  

- Tipo: categórica ordinal   
- Valores posibles: alto, medio, bajo  
- Transformación: codificación ordinal con asignación:  bajo -> 0, medio -> 1, alto -> 2  



**otras_enfermedades**:   

- Tipo: categórica  nominal  
- Transformación: dummy variables  


#### Vamos a guardar los valores de cada una de las variables ordinales para no tener que utilizar X_todos:


In [23]:
valores_envergadura=['pequeño', 'medio', 'grande']  
valores_globulos_blancos=["bajo", "normal", "alto"]
valores_altura=["alto", "medio", "bajo" ]
valores_otras_enfermedades=["No", "Si"]


#### Transformadores de variables

In [24]:
# importamos las librerías necesarias
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

#### Transformador/es para  **variables nominales** con OneHotEncoder

In [25]:
# Dummy para otras_enfermedades:
t_nominal=("onehot",OneHotEncoder(sparse=False,categories=[valores_otras_enfermedades]),['otras_enfermedades'])

#### Transformador/es para variables Ordinales

En este caso tendremos que usar dos, porque algunas variables queremos que se codifiquen en forma automática y a otras les forzaremos u orden que deseamos:

In [26]:
# Ordinal con asignación de orden para envergadura, altura y globulos_blancos
t_ordinal_enc=("ordinal_enc", OrdinalEncoder(categories=[valores_envergadura,valores_altura, valores_globulos_blancos]), 
               ['envergadura','altura','globulos_blancos'])

#### Creamos Transformador de Columnas

In [27]:
# importamos librería
from sklearn.compose import ColumnTransformer

Le pasamos todos los transformadores:

In [28]:
transformador_columnas_categoricas = ColumnTransformer(transformers=[t_nominal, t_ordinal_enc],
                                                       remainder='passthrough')


#### Entrenamos con fit en X_train

In [29]:
transformador_columnas_categoricas.fit(X_train)
;  

' '

#### Aplicamos el Transformador de Columnas a X_train y X_test para obtener X_train_transformado y X_test_transformado respectivamente

In [30]:
X_train_transformado=transformador_columnas_categoricas.transform(X_train)
X_train_transformado

array([[ 0.,  1.,  0.,  0.,  1., 50.],
       [ 1.,  0.,  1.,  1.,  2., 73.],
       [ 1.,  0.,  1.,  2.,  1., 68.],
       [ 0.,  1.,  0.,  0.,  2., 65.],
       [ 0.,  1.,  1.,  1.,  1., 76.],
       [ 1.,  0.,  1.,  1.,  1., 77.],
       [ 0.,  1.,  0.,  2.,  2., 52.]])

In [31]:
X_test_transformado=transformador_columnas_categoricas.transform(X_test)
X_test_transformado

array([[ 1.,  0.,  2.,  1.,  2., 93.],
       [ 1.,  0.,  1.,  2.,  1., 65.],
       [ 0.,  1.,  0.,  0.,  2., 43.],
       [ 0.,  1.,  2.,  1.,  0., 86.]])

Listo!, ya están el X_train y X_test listos para pasar a algún algoritmo de ML. 